# ResNET
Experiments using Resnet models and fine-tuning with my usecase. 

In [1]:
from pathlib import Path

import pytorch_lightning as pl
import torch.nn as nn
import torchmetrics as tm
import torchvision.models as models

from torch.optim import SGD, Adam
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

class ResNetClassifier(pl.LightningModule):
    resnets = {
        18: models.resnet18,
        34: models.resnet34,
        50: models.resnet50,
        101: models.resnet101,
        152: models.resnet152,
    }
    
    optimizers = {"adam": Adam, "sgd": SGD}

    def __init__(
        self,
        num_classes,
        resnet_version,
        train_path,
        val_path,
        test_path=None,
        optimizer="adam",
        lr=1e-3,
        batch_size=16,
        tune_fc_only=True,
    ):
        super().__init__()

        self.num_classes = num_classes
        self.train_path = train_path
        self.val_path = val_path
        self.test_path = test_path
        self.lr = lr
        self.batch_size = batch_size

        self.optimizer = self.optimizers[optimizer]
        self.loss_fn = (nn.BCEWithLogitsLoss() if num_classes == 1 else nn.CrossEntropyLoss())
        # Metrics
        self.accuracy = tm.Accuracy(task="binary").to(self.device)
        
        # Using a pretrained ResNet backbone
        self.resnet_model = self.resnets[resnet_version]()
        
        # Replace old FC layer with Identity to train own
        linear_size = list(self.resnet_model.children())[-1].in_features
        
        # replace final layer for fine-tuning
        self.resnet_model.fc = nn.Linear(linear_size, num_classes)

        if tune_fc_only:  # option to only tune the fully-connected layers
            for child in list(self.resnet_model.children())[:-1]:
                for param in child.parameters():
                    param.requires_grad = False
        
        self.save_hyperparameters()
        
    def forward(self, X):
        return self.resnet_model(X)

    def configure_optimizers(self):
        return self.optimizer(self.parameters(), lr=self.lr)

    def _step(self, batch):
        x, y = batch
        preds = self(x)

        if self.num_classes == 1:
            preds = preds.flatten()
            y = y.float()

        loss = self.loss_fn(preds, y)
        acc = self.accuracy(preds, y)
        return loss, acc

    def _dataloader(self, data_path, shuffle=False):
        transform = transforms.Compose(
            [
                transforms.Resize((256, 256)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ]
        )
        img_folder = ImageFolder(data_path, transform=transform)

        return DataLoader(img_folder, batch_size=self.batch_size, shuffle=shuffle, num_workers=8, persistent_workers=True)

    def train_dataloader(self):
        return self._dataloader(self.train_path, shuffle=True)

    def training_step(self, batch, batch_idx):
        loss, acc = self._step(batch)
        
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("train_acc", acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def val_dataloader(self):
        return self._dataloader(self.val_path)

    def validation_step(self, batch, batch_idx):
        loss, acc = self._step(batch)

        self.log("val_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        self.log("val_acc", acc, on_epoch=True, prog_bar=True, logger=True)

    def test_dataloader(self):
        return self._dataloader(self.test_path)

    def test_step(self, batch, batch_idx):
        loss, acc = self._step(batch)

        self.log("test_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        self.log("test_acc", acc, on_epoch=True, prog_bar=True, logger=True)

In [2]:
import os

from dotenv import load_dotenv
load_dotenv()
root_data = os.getenv("KAGGLE_FILES_DIR")
dataset_path = Path(os.getcwd(), "..", root_data, 'processed')

In [63]:
resnet18 = ResNetClassifier(
    num_classes=1,
    resnet_version=18,
    train_path=Path(dataset_path, "train"),
    val_path=Path(dataset_path, "val"),
    test_path=Path(dataset_path, "test"),
    optimizer="adam",
    lr=1e-4,
    batch_size=32,
    tune_fc_only=False,
)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath="../models/resnet18",
    filename="resnet18-model-{epoch}-{val_loss:.3f}-{val_acc:0.3f}",
    monitor="val_loss",
    save_top_k=2,
    mode="min",
    save_last=True,
)

early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=True,
    mode='min'
)

# Instantiate lightning trainer and train model
trainer_args = {
    "accelerator": "mps",
    "max_epochs": 100,
    "callbacks": [checkpoint_callback, early_stopping],
    "precision": 32,
}
trainer = pl.Trainer(**trainer_args)



GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [64]:
trainer.fit(resnet18)


  | Name         | Type                  | Params
-------------------------------------------------------
0 | loss_fn      | BCEWithLogitsLoss     | 0     
1 | accuracy     | BinaryAccuracy        | 0     
2 | precision    | BinaryPrecision       | 0     
3 | recall       | BinaryRecall          | 0     
4 | auc          | BinaryAUROC           | 0     
5 | f1           | BinaryF1Score         | 0     
6 | conf         | BinaryConfusionMatrix | 0     
7 | resnet_model | ResNet                | 11.2 M
-------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.708    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.555


...

Metric val_loss improved by 0.010 >= min_delta = 0.001. New best score: 0.470


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.470. Signaling Trainer to stop.


In [65]:
trainer.test(resnet18)


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc             0.775600790977478
        test_auc           0.0028453900013118982
         test_f1                    nan
        test_loss           0.5288562774658203
        test_prec                   nan
        test_rec                    nan
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.5288562774658203,
  'test_acc': 0.775600790977478,
  'test_prec': nan,
  'test_rec': nan,
  'test_f1': nan,
  'test_auc': 0.0028453900013118982}]

Best ResNet18 has loss 0.47 and accuracy 0.80
Next is ResNet34.


# ResNet34

In [70]:
resnet34 = ResNetClassifier(
    num_classes=1,
    resnet_version=34,
    train_path=Path(dataset_path, "train"),
    val_path=Path(dataset_path, "val"),
    test_path=Path(dataset_path, "test"),
    optimizer="adam",
    lr=1e-4,
    batch_size=32,
    tune_fc_only=False,
)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath="../models/resnet34",
    filename="resnet34-model-{epoch}-{val_loss:.3f}-{val_acc:0.3f}",
    monitor="val_loss",
    save_top_k=2,
    mode="min",
    save_last=True,
)

early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=True,
    mode='min'
)

# Instantiate lightning trainer and train model
trainer_args = {
    "accelerator": "mps",
    "max_epochs": 100,
    "callbacks": [checkpoint_callback, early_stopping],
    "precision": 32,
}
trainer = pl.Trainer(**trainer_args)



GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [71]:
trainer.fit(resnet34)


  | Name         | Type                  | Params
-------------------------------------------------------
0 | loss_fn      | BCEWithLogitsLoss     | 0     
1 | accuracy     | BinaryAccuracy        | 0     
2 | precision    | BinaryPrecision       | 0     
3 | recall       | BinaryRecall          | 0     
4 | auc          | BinaryAUROC           | 0     
5 | f1           | BinaryF1Score         | 0     
6 | conf         | BinaryConfusionMatrix | 0     
7 | resnet_model | ResNet                | 21.3 M
-------------------------------------------------------
21.3 M    Trainable params
0         Non-trainable params
21.3 M    Total params
85.141    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.561


...

Metric val_loss improved by 0.002 >= min_delta = 0.001. New best score: 0.460


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.460. Signaling Trainer to stop.


The best ResNet34 model has loss 0.460 and accuracy 0.803 - it is slightly better than ResNet18, but not much.

# ResNet50

In [3]:
resnet50 = ResNetClassifier(
    num_classes=1,
    resnet_version=50,
    train_path=Path(dataset_path, "train"),
    val_path=Path(dataset_path, "val"),
    test_path=Path(dataset_path, "test"),
    optimizer="adam",
    lr=1e-4,
    batch_size=32,
    tune_fc_only=False,
)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath="../models/resnet50",
    filename="resnet50-model-{epoch}-{val_loss:.3f}-{val_acc:0.3f}",
    monitor="val_loss",
    save_top_k=2,
    mode="min",
    save_last=True,
)
early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=True,
    mode='min'
)

# Instantiate lightning trainer and train model
trainer_args = {
    "accelerator": "mps",
    "max_epochs": 100,
    "callbacks": [checkpoint_callback, early_stopping],
    "precision": 32,
}
trainer = pl.Trainer(**trainer_args)



GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
trainer.fit(resnet50)


  | Name         | Type              | Params
---------------------------------------------------
0 | loss_fn      | BCEWithLogitsLoss | 0     
1 | accuracy     | BinaryAccuracy    | 0     
2 | resnet_model | ResNet            | 23.5 M
---------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.040    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.568


...

Metric val_loss improved by 0.011 >= min_delta = 0.001. New best score: 0.457


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.457. Signaling Trainer to stop.


In [7]:
model = ResNetClassifier.load_from_checkpoint(checkpoint_callback.best_model_path)

In [8]:
trainer.test(model)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8041784167289734     │
│         test_loss         │    0.4585435092449188     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.4585435092449188, 'test_acc': 0.8041784167289734}]

No more models will be tested. Gains are minimal, and training time grow exponentially.
I will use best ResNet50 model for implementing API.